In [ ]:
pip install insightface opencv-python numpy
pip install onnxruntime
# pip install onnxruntime-gpu for gpu
sudo apt update
sudo apt install build-essential
sudo apt install python3-dev

SyntaxError: invalid syntax (854577940.py, line 1)

In [ ]:
sudo apt update
sudo apt install libgtk2.0-dev pkg-config
pip uninstall opencv-python opencv-contrib-python -y
pip install opencv-python opencv-contrib-python


In [4]:
import cv2
import os
import time
import numpy as np
from insightface.app import FaceAnalysis

In [5]:
import cv2
import os

# ======================
# Configuration
# ======================
USER_NAME = "user1"         # change per user
SAVE_DIR = "dataset"
TOTAL_PER_POSE = 5
poses = ["front", "left", "right"]

# Create directories for each pose
for pose in poses:
    os.makedirs(os.path.join(SAVE_DIR, USER_NAME, pose), exist_ok=True)

# Load Haarcascade for face detection
face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

# ======================
# Countdown function
# ======================
def countdown(cap):
    for i in range(3, 0, -1):
        ret, frame = cap.read()
        if not ret:
            continue
        cv2.putText(frame, str(i), (260, 250),
                    cv2.FONT_HERSHEY_SIMPLEX, 4, (0, 0, 255), 4)
        cv2.imshow("Capture", frame)
        cv2.waitKey(1000)

# ======================
# Draw face borders function
# ======================
def draw_face_borders(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 0), 3)  # black border
    return frame, faces

# ======================
# Wait for 'S' function
# ======================
def wait_for_s(cap, message):
    while True:
        ret, frame = cap.read()
        if not ret:
            continue

        # Draw face border in live preview
        frame, _ = draw_face_borders(frame)

        cv2.putText(frame, message, (50, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 255), 2)
        cv2.putText(frame, "Press S to start | Q to quit", (50, 90),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

        cv2.imshow("Capture", frame)
        key = cv2.waitKey(1) & 0xFF

        if key == ord('s'):
            return True
        elif key == ord('q'):
            return False

# ======================
# Initialize camera
# ======================
cap = cv2.VideoCapture(0)

for pose in poses:

    ok = wait_for_s(cap, f"Ready for {pose.upper()} pose")
    if not ok:
        break

    countdown(cap)

    saved = 0
    while saved < TOTAL_PER_POSE:
        ret, frame = cap.read()
        if not ret:
            continue

        # Draw face border
        frame, faces = draw_face_borders(frame)

        # Instruction overlay
        cv2.putText(frame, f"{pose.upper()} {saved}/{TOTAL_PER_POSE}", (30, 450),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 0), 2)

        if len(faces) == 0:
            # Show message and stop pose capture
            cv2.putText(frame, "No face detected! Move to correct position.", (50, 200),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
            cv2.imshow("Capture", frame)
            cv2.waitKey(1500)  # pause to show message
            print(f"No face detected for {pose.upper()}, stopping this pose.")
            break

        # If face detected, crop and save
        (x, y, w, h) = faces[0]  # take first detected face
        face_img = frame[y:y+h, x:x+w]
        filename = os.path.join(SAVE_DIR, USER_NAME, pose, f"{saved+1}.jpg")
        cv2.imwrite(filename, face_img)
        saved += 1

        cv2.imshow("Capture", frame)

        # small delay between captures
        if cv2.waitKey(400) & 0xFF == ord('q'):
            cap.release()
            cv2.destroyAllWindows()
            exit()

print("✅ Face capture completed!")

# ======================
# Finish screen
# ======================
while True:
    ret, frame = cap.read()
    if not ret:
        continue

    cv2.putText(frame, "Capture Successful!", (150, 200),
                cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 0), 3)
    cv2.putText(frame, "Press Q to Quit", (180, 260),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)

    cv2.imshow("Capture", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


✅ Face capture completed!


In [6]:

USER_NAME = "user1"  # change per user
SAVE_DIR = "embeddings"
OUTPUT_FILE = os.path.join(SAVE_DIR, f"{USER_NAME}_embedding.npy")
TOTAL_PER_POSE = 5
poses = ["front", "left", "right"]

# Create directories for storing embeddings
os.makedirs(SAVE_DIR, exist_ok=True)

# Load Haarcascade for face detection
face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

# Initialize InsightFace for embedding extraction
app = FaceAnalysis(name="buffalo_l")  # MobileFaceNet ArcFace
app.prepare(ctx_id=-1)  # CPU: -1, GPU: 0

# Store all embeddings (across all poses)
all_embeddings = []


# ======================
# Countdown function
# ======================
def countdown(cap):
    for i in range(3, 0, -1):
        ret, frame = cap.read()
        if not ret:
            continue
        cv2.putText(
            frame, str(i), (260, 250), cv2.FONT_HERSHEY_SIMPLEX, 4, (0, 0, 255), 4
        )
        cv2.imshow("Capture", frame)
        cv2.waitKey(1000)


# ======================
# Get face embedding from frame
# ======================
def get_embedding_from_frame(frame):
    """Extract face embedding from frame"""
    faces = app.get(frame)
    if len(faces) == 0:
        return None
    return faces[0].embedding  # 512-d vector


# ======================
# Face Detection Preview
# ======================
def face_detection_preview(cap):
    """Show face detection preview with rectangle or 'no face detected' message"""
    while True:
        ret, frame = cap.read()
        if not ret:
            continue

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(
            gray, scaleFactor=1.1, minNeighbors=5, minSize=(100, 100)
        )

        # Show instruction
        cv2.putText(
            frame,
            "Position your face in the center",
            (50, 50),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.8,
            (0, 255, 255),
            2,
        )
        cv2.putText(
            frame,
            "Press S to continue | Q to quit",
            (50, 90),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.7,
            (0, 255, 0),
            2,
        )

        if len(faces) > 0:
            # Face detected - draw black rectangle
            (x, y, w, h) = faces[0]
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 0), 3)
            cv2.putText(
                frame,
                "Face Detected!",
                (50, 130),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.8,
                (0, 255, 0),
                2,
            )
        else:
            # No face detected
            cv2.putText(
                frame,
                "No face detected",
                (50, 130),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.8,
                (0, 0, 255),
                2,
            )

        cv2.imshow("Capture", frame)
        key = cv2.waitKey(1) & 0xFF

        if key == ord("s"):
            return True
        elif key == ord("q"):
            return False


# ======================
# Initialize camera
# ======================
cap = cv2.VideoCapture(0)

# Show face detection preview first
ok = face_detection_preview(cap)
if not ok:
    cap.release()
    cv2.destroyAllWindows()
    exit()

total_captured = 0

for pose in poses:

    # Show face detection preview for this pose
    ok = face_detection_preview(cap)
    if not ok:
        break

    countdown(cap)

    saved = 0
    
    while saved < TOTAL_PER_POSE:
        ret, frame = cap.read()
        if not ret:
            continue

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)

        # Instruction overlay
        cv2.putText(
            frame,
            f"{pose.upper()} {saved}/{TOTAL_PER_POSE} (Total: {total_captured + saved + 1}/{len(poses) * TOTAL_PER_POSE})",
            (30, 450),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.8,
            (255, 255, 0),
            2,
        )

        if len(faces) == 0:
            # No face detected - show message and continue waiting
            cv2.putText(
                frame,
                "No face detected! Move to correct position.",
                (50, 200),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.8,
                (0, 0, 255),
                2,
            )
            cv2.imshow("Capture", frame)
            if cv2.waitKey(400) & 0xFF == ord("q"):
                cap.release()
                cv2.destroyAllWindows()
                exit()
            continue

        # If face detected, draw black outline rectangle
        (x, y, w, h) = faces[0]  # take first detected face
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 0), 3)

        cv2.imshow("Capture", frame)

        # Extract embedding from frame
        embedding = get_embedding_from_frame(frame)
        if embedding is None:
            print(f"Failed to extract embedding for {pose.upper()}")
            continue

        # Store embedding
        all_embeddings.append(embedding)
        saved += 1
        total_captured += 1
        print(f"Captured {pose.upper()} embedding {saved}/{TOTAL_PER_POSE}")

        # small delay between captures
        if cv2.waitKey(400) & 0xFF == ord("q"):
            cap.release()
            cv2.destroyAllWindows()
            exit()

print("✅ Face embedding capture completed!")

# Average all embeddings from all poses into a single embedding
if len(all_embeddings) > 0:
    combined_embedding = np.mean(all_embeddings, axis=0)
    combined_embedding /= np.linalg.norm(combined_embedding)  # normalize
    np.save(OUTPUT_FILE, combined_embedding)
    print(f"✅ Combined embedding saved to {OUTPUT_FILE}")
    print(f"   Total embeddings captured: {len(all_embeddings)}")
    print(f"   Embedding shape: {combined_embedding.shape}")

# ======================
# Finish screen
# ======================
while True:
    ret, frame = cap.read()
    if not ret:
        continue

    cv2.putText(
        frame,
        "Capture Successful!",
        (150, 200),
        cv2.FONT_HERSHEY_SIMPLEX,
        1.2,
        (0, 255, 0),
        3,
    )
    cv2.putText(
        frame,
        "Press Q to Quit",
        (180, 260),
        cv2.FONT_HERSHEY_SIMPLEX,
        1,
        (0, 255, 255),
        2,
    )

    cv2.imshow("Capture", frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/nimesh/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/nimesh/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/nimesh/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/nimesh/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/nimesh/.insightface/models/buffalo_l/w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
se

In [8]:
embedding = np.load("embeddings/user2_embedding.npy")
embedding

array([-0.00340365, -0.01121106,  0.01984171, -0.02328454,  0.00929982,
        0.08641303,  0.07128216,  0.00627419,  0.01166763,  0.04897914,
        0.05759859, -0.01680785, -0.0196371 ,  0.01670406,  0.03560349,
       -0.02941946,  0.05668332,  0.01748371,  0.07780635,  0.05108226,
        0.09288155, -0.07323273,  0.01998658, -0.06879342,  0.00289742,
       -0.04825173, -0.05305039, -0.05051847,  0.04078783, -0.02272603,
       -0.05094613, -0.03581465,  0.04685441, -0.01914132,  0.0144848 ,
       -0.02831838, -0.04956057,  0.02584866,  0.00907941, -0.03994212,
       -0.05864436,  0.01206964, -0.00352414,  0.00500929,  0.03488876,
       -0.04004527,  0.02467619, -0.02072682, -0.10129276, -0.01152482,
       -0.00796311,  0.03564569, -0.00997269, -0.05422456,  0.02344665,
       -0.04600961,  0.03168477, -0.02894596, -0.07201705, -0.05263985,
        0.00638105, -0.04468837, -0.05414935,  0.04514595, -0.06599743,
        0.01392351, -0.08108491,  0.03002172,  0.03818974,  0.03

In [1]:
import cv2
import os
import numpy as np
from insightface.app import FaceAnalysis

In [ ]:


# ======================
# Configuration
# ======================
USER_NAME = "user1"  # change per user
SAVE_DIR = "embeddings"
OUTPUT_FILE = os.path.join(SAVE_DIR, f"{USER_NAME}_embedding.npy")
TOTAL_PER_POSE = 5
poses = ["front", "left", "right"]

os.makedirs(SAVE_DIR, exist_ok=True)

# Initialize InsightFace (RetinaFace + ArcFace)
app = FaceAnalysis(name="buffalo_l")  # detection + recognition
app.prepare(ctx_id=-1)  # CPU; use 0 for GPU

# Store all embeddings (across all poses)
all_embeddings = []

# ======================
# Countdown function
# ======================
def countdown(cap):
    for i in range(3, 0, -1):
        ret, frame = cap.read()
        if not ret:
            continue
        cv2.putText(frame, str(i), (260, 250), cv2.FONT_HERSHEY_SIMPLEX, 4, (0, 0, 255), 4)
        cv2.imshow("Capture", frame)
        cv2.waitKey(1000)


# ======================
# Get face embedding from frame
# ======================
def get_embedding_from_frame(frame):
    """Extract face embedding from frame using RetinaFace"""
    faces = app.get(frame)
    if len(faces) == 0:
        return None, None
    face = faces[0]
    return face.embedding, face.bbox  # 512-d vector, bounding box


# ======================
# Face Detection Preview
# ======================
def face_detection_preview(cap):
    """Show preview until user presses S"""
    while True:
        ret, frame = cap.read()
        if not ret:
            continue

        faces = app.get(frame)
        if len(faces) > 0:
            x1, y1, x2, y2 = map(int, faces[0].bbox)
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 0), 3)
            cv2.putText(frame, "Face Detected!", (50, 130), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
        else:
            cv2.putText(frame, "No face detected", (50, 130), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)

        cv2.putText(frame, "Position your face in the center", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)
        cv2.putText(frame, "Press S to continue | Q to quit", (50, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

        cv2.imshow("Capture", frame)
        key = cv2.waitKey(1) & 0xFF
        if key == ord("s"):
            return True
        elif key == ord("q"):
            return False


# ======================
# Initialize camera
# ======================
cap = cv2.VideoCapture(0)

# Show preview
ok = face_detection_preview(cap)
if not ok:
    cap.release()
    cv2.destroyAllWindows()
    exit()

total_captured = 0

for pose in poses:
    ok = face_detection_preview(cap)
    if not ok:
        break

    countdown(cap)

    saved = 0
    while saved < TOTAL_PER_POSE:
        ret, frame = cap.read()
        if not ret:
            continue

        embedding, bbox = get_embedding_from_frame(frame)
        if embedding is None:
            cv2.putText(frame, "No face detected! Move to correct position.", (50, 200), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
            cv2.imshow("Capture", frame)
            if cv2.waitKey(400) & 0xFF == ord("q"):
                cap.release()
                cv2.destroyAllWindows()
                exit()
            continue

        x1, y1, x2, y2 = map(int, bbox)
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 0), 3)
        cv2.imshow("Capture", frame)

        # Store embedding
        all_embeddings.append(embedding)
        saved += 1
        total_captured += 1
        print(f"Captured {pose.upper()} embedding {saved}/{TOTAL_PER_POSE}")

        if cv2.waitKey(400) & 0xFF == ord("q"):
            cap.release()
            cv2.destroyAllWindows()
            exit()

print("✅ Face embedding capture completed!")

# Average all embeddings into one vector
if len(all_embeddings) > 0:
    combined_embedding = np.mean(all_embeddings, axis=0)
    combined_embedding /= np.linalg.norm(combined_embedding)
    np.save(OUTPUT_FILE, combined_embedding)
    print(f"✅ Combined embedding saved to {OUTPUT_FILE}")
    print(f"Total embeddings captured: {len(all_embeddings)}")
    print(f"Embedding shape: {combined_embedding.shape}")

# Finish screen
while True:
    ret, frame = cap.read()
    if not ret:
        continue

    cv2.putText(frame, "Capture Successful!", (150, 200), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 0), 3)
    cv2.putText(frame, "Press Q to Quit", (180, 260), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
    cv2.imshow("Capture", frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


/home/nimesh/anaconda3/envs/nimesh/lib/python3.9/site-packages/onnxruntime/capi/onnxruntime_inference_collection.py:69: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/nimesh/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/nimesh/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/nimesh/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/nimesh/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/nimesh/.insightface/models/buffalo_l/w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
se

QFontDatabase: Cannot find font directory /home/nimesh/anaconda3/envs/nimesh/lib/python3.9/site-packages/cv2/qt/fonts.
Note that Qt no longer ships fonts. Deploy some (from https://dejavu-fonts.github.io/ for example) or switch to fontconfig.
QFontDatabase: Cannot find font directory /home/nimesh/anaconda3/envs/nimesh/lib/python3.9/site-packages/cv2/qt/fonts.
Note that Qt no longer ships fonts. Deploy some (from https://dejavu-fonts.github.io/ for example) or switch to fontconfig.
QFontDatabase: Cannot find font directory /home/nimesh/anaconda3/envs/nimesh/lib/python3.9/site-packages/cv2/qt/fonts.
Note that Qt no longer ships fonts. Deploy some (from https://dejavu-fonts.github.io/ for example) or switch to fontconfig.
QFontDatabase: Cannot find font directory /home/nimesh/anaconda3/envs/nimesh/lib/python3.9/site-packages/cv2/qt/fonts.
Note that Qt no longer ships fonts. Deploy some (from https://dejavu-fonts.github.io/ for example) or switch to fontconfig.
QFontDatabase: Cannot find f

KeyboardInterrupt: 

: 

In [ ]:
from scipy.spatial.distance import cosine

# ==========================
# Config
# ==========================
EMBEDDINGS_DIR = "embeddings"
THRESHOLD = 0.5  # cosine distance threshold for recognition

# Load saved embeddings
user_embeddings = {}
for file in os.listdir(EMBEDDINGS_DIR):
    if file.endswith(".npy"):
        name = file.split("_")[0]  # e.g., "user1_embedding.npy"
        emb = np.load(os.path.join(EMBEDDINGS_DIR, file))
        user_embeddings[name] = emb

# Load ArcFace with RetinaFace for detection
app = FaceAnalysis(name="buffalo_l")  # detection + recognition
app.prepare(ctx_id=-1)  # CPU; use 0 for GPU

# Initialize webcam
cap = cv2.VideoCapture(0)

print("Real-time face recognition started. Press Q to quit.")

while True:
    ret, frame = cap.read()
    if not ret:
        continue

    # Detect faces using RetinaFace
    faces = app.get(frame)  # returns list of face objects

    for face in faces:
        x1, y1, x2, y2 = map(int, face.bbox)  # bounding box
        embedding = face.embedding / np.linalg.norm(face.embedding)

        # Compare with stored embeddings
        best_match = "Unknown"
        best_score = 1.0
        for name, stored_emb in user_embeddings.items():
            score = cosine(embedding, stored_emb)
            if score < best_score:
                best_score = score
                best_match = name

        if best_score > THRESHOLD:
            best_match = "Unknown"

        # Draw bounding box and label
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, f"{best_match} ({best_score:.2f})", (x1, y1-10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
        cv2.putText(frame, "Press Q to Quit", (35,35), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
        

    # Show main frame
    cv2.imshow("Face Recognition", frame)
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break

# Cleanup
cap.release()
cv2.destroyAllWindows()


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/nimesh/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/nimesh/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/nimesh/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/nimesh/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/nimesh/.insightface/models/buffalo_l/w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
se

In [3]:

# ======================
# Config
# ======================
DATASET_DIR = "dataset"
OUTPUT_DIR = "embeddings"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Load ArcFace model
app = FaceAnalysis(name="buffalo_l")  # MobileFaceNet ArcFace
app.prepare(ctx_id=-1)  # CPU: -1, GPU: 0

# ======================
# Helper to get embedding from image
# ======================
def get_embedding(image_path):
    img = cv2.imread(image_path)
    if img is None:
        return None
    faces = app.get(img)
    if len(faces) == 0:
        print(f"No face detected in {image_path}")
        return None
    return faces[0].embedding  # 512-d vector

# ======================
# Process dataset
# ======================
for user_name in os.listdir(DATASET_DIR):
    user_path = os.path.join(DATASET_DIR, user_name)
    if not os.path.isdir(user_path):
        continue

    print(f"\nProcessing {user_name}")
    user_embeddings = {}

    for pose in ["front", "left", "right"]:
        pose_path = os.path.join(user_path, pose)
        if not os.path.isdir(pose_path):
            continue

        embeddings_list = []
        for img_name in os.listdir(pose_path):
            img_path = os.path.join(pose_path, img_name)
            emb = get_embedding(img_path)
            if emb is not None:
                embeddings_list.append(emb)

        if len(embeddings_list) == 0:
            print(f"No valid face images for {user_name} {pose}")
            continue

        # Average embeddings for this pose
        avg_embedding = np.mean(embeddings_list, axis=0)
        avg_embedding /= np.linalg.norm(avg_embedding)  # normalize

        user_embeddings[pose] = avg_embedding

    # Save embeddings for this user
    out_file = os.path.join(OUTPUT_DIR, f"{user_name}_embeddings.npz")
    np.savez(out_file, **user_embeddings)
    print(f"Saved embeddings for {user_name} -> {out_file}")


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/nimesh/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/nimesh/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/nimesh/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/nimesh/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/nimesh/.insightface/models/buffalo_l/w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
se